# Check Register Handling


In [1]:
import pandas as pd
import numpy as np
from config import config

loading environment variables
cwd = D:\Coding\Finance-Model\notebooks
inside of notebook directory
   setting new working directory: D:\Coding\Finance-Model


In [2]:
peachtree_df = pd.ExcelFile(config['peach_tree_checks'])

peachtree_df = pd.read_excel(peachtree_df, dtype={"Check #": str,
                                                  "Cash Account": str})

# Improve this
peachtree_df = peachtree_df.iloc[:len(peachtree_df) - 2, :]
print(peachtree_df.dtypes)
peachtree_df.loc[peachtree_df["Amount"].isna(), "Amount"] = 0

Check #                 object
Date            datetime64[ns]
Payee                   object
Cash Account            object
Amount                 float64
dtype: object


In [6]:


# Process checks that were voided after issue
void_mask = peachtree_df["Check #"].str.endswith("V")
check_no = peachtree_df.loc[void_mask, "Check #"].str.rstrip("V")


void_df = pd.DataFrame({'Check #': peachtree_df.loc[void_mask, "Check #"].str.rstrip("V"),
                        'Void': True,
                        'Void Date': peachtree_df.loc[void_mask, "Date"]
                        })
merged_df = pd.merge(peachtree_df, void_df, how='left')
merged_df.loc[merged_df['Void'].isna(), 'Void'] = False

online_mask = merged_df["Check #"].str.startswith("ON LINE PAY")
check_mask = merged_df["Check #"].str.len() == 5
check_df = merged_df[check_mask]

void_mask = peachtree_df["Amount"] == 0
check_df.loc[void_mask, "Amount"] = 0
check_df.loc[void_mask, "Void"] = True
check_df.loc[void_mask, "Void Date"] = check_df.loc[void_mask, "Date"]

good_check_mask = check_df["Void"] == False
good_check_df = check_df[good_check_mask]

account_number = '5559464829'
# positive_pay_df = pd.DataFrame({'Account #'})
# peachtree_df['void'] = False
# peachtree_df['void'][a] = True
# voided = peachtree_df[a]
# voided["Void Check"] = voided["Check #"]
# 

# voided["voided"] = voided["Check #"].str.rstrip("V")
#voided.loc[:, "Check #"] = check_no

In [7]:
good_check_df
good_check_df.dtypes
check = good_check_df.loc[1, :]
check
check_str = f'{account_number}{check["Check #"]:0>10}{check["Amount"]:11.2f}{check["Date"].month:02d}{check["Date"].day:02d}{check["Date"].year}{check["Payee"]:<50}{" ":<50}'
len(check_str)

# Normal format
with open(f'{config['processed_dir']}/ISSUEPPPFW.txt', 'w') as f:
    for index, check in good_check_df.iterrows():
        check_str = f'{account_number}{check["Check #"]:0>10}{check["Amount"]:11.2f}{check["Date"].month:02d}{check["Date"].day:02d}{check["Date"].year}{check["Payee"]:<50}{" ":<50}'
        f.write(check_str + '\n')
        
with open(f'{config['processed_dir']}/ISSUEVOIDFW.txt', 'w') as f:
    for index, check in check_df.iterrows():
        if check["Void"]:
            action = "V"
        else:
            action = "I"
        check_str = f'{account_number}{check["Check #"]:0>10}{check["Amount"]:11.2f}{check["Date"].month:02d}{check["Date"].day:02d}{check["Date"].year}{" ":<15}{check["Payee"]:<50}{" ":<50}{action}'
        f.write(check_str + '\n')

---
## Sample Code

In [17]:
data1 = {'Code': [1, 2, 3],
         'Name': ['Company1', 'Company2', 'Company3'],
         'Value': [200, 300, 400], 
         'Extra Value': [17, 18, 19]
    }
df1 = pd.DataFrame(data1) #, columns= ['Code','Name','Value'])

data2 = {
         'Code': [2],
         'Name': ['Company2'],
         'Value': [1000],
    }

df2 = pd.DataFrame(data2) #, columns= ['Code','Name','Value'])

In [19]:
updated = df1.merge(df2, how='left', on='Code', suffixes=['', '_new'])
updated

,Code,Name,Value,Extra Value,Name_new,Value_new
0,1,Company1,200,17,NaN,NaN
1,2,Company2,300,18,Company2,1000.0
2,3,Company3,400,19,NaN,NaN


In [24]:
updated['Value'] = np.where(pd.notnull(updated['Value_new']), updated['Value_new'], updated['Value'])

In [25]:
updated

,Code,Name,Value,Extra Value,Name_new,Value_new
0,1,Company1,200.0,17,NaN,NaN
1,2,Company2,1000.0,18,Company2,1000.0
2,3,Company3,400.0,19,NaN,NaN


In [16]:
df1.index

Index([1, 2, 3], dtype='int64', name='Code')